# Install packages

In [ ]:
!pip install scenic
!pip install einops
# !pip install -qq medmnist
!pip install --upgrade keras
!pip install keras-tuner
!pip install gdown

In [2]:
import tensorflow as tf
import os
import keras
from keras import layers, ops, regularizers

import numpy as np
# import matplotlib.pyplot as plt
# import skvideo.io
import pandas as pd
import cv2
import os

# Reload data from pkl

#### Video

In [ ]:
import pickle
with open('/kaggle/input/3july-2-pkll/3July_2/ab_videos.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
ab_videos=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    ab_videos.append(array_a_loaded)
  except:
    print('end!')
    break

In [ ]:
with open('/kaggle/input/3july-2-pkll/3July_2/nor_videos.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
nor_videos=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    nor_videos.append(array_a_loaded)
  except:
    print('end!')
    break

#### Dims

In [ ]:
with open('/kaggle/input/3july-2-pkll/3July_2/ab_video_dims.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
ab_video_dims=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    ab_video_dims.append(array_a_loaded)
  except:
    print('end!')
    break
with open('/kaggle/input/3july-2-pkll/3July_2/nor_video_dims.pkl', 'rb') as file:
    loaded_vid_list = pickle.load(file)
nor_video_dims=[]
x=0
while True:
  try:
    array_a_loaded = loaded_vid_list[x]
    x+=1
    nor_video_dims.append(array_a_loaded)
  except:
    print('end!')
    break

# Modelling pre-process <a class="anchor"  id="modelpre"></a>

In [ ]:
human_action_dataset = np.asarray( ab_videos + nor_videos )

In [ ]:
# abnormal = 1
# normal = 0
labels = np.concatenate([np.ones(len(ab_videos)),np.zeros(len(nor_videos))])

In [ ]:
ab_videos_dims_df = pd.DataFrame(ab_video_dims,columns=['frames','width','height','channel'])
nor_videos_dims_df = pd.DataFrame(nor_video_dims,columns=['frames','width','height','channel'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(human_action_dataset,labels,test_size=0.2,shuffle=True,random_state=42,stratify=labels)
X_test,X_valid,y_test,y_valid = train_test_split(X_test,y_test,test_size=0.5,shuffle=True,random_state=42)

In [4]:
# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

# Trans body

In [12]:
# Setting seed for reproducibility
SEED = 77
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
tf.random.set_seed(SEED)

# DATA
BATCH_SIZE = 4
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (40, 224, 224, 3)
NUM_CLASSES = 2

# OPTIMIZER
# LEARNING_RATE = 1e-4 #Default
LEARNING_RATE = 1e-4 #CHANGE THIS
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 20

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 64
NUM_HEADS = 2
NUM_LAYERS = 2
# NUM_HEADS = 4
# NUM_LAYERS = 4

In [ ]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """
      Preprocess the frames tensors and parse the labels.
      This code convert the framess into float for normalization hence
      we should add a convert dtype code back to the model
    """
    # Preprocess images
    frames = tf.image.convert_image_dtype(frames[..., tf.newaxis],  tf.float32,)
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

# Training set
trainloader = prepare_dataloader(X_train, y_train, "train")
testloader = prepare_dataloader(X_test, y_test, "test")
validloader = prepare_dataloader(X_valid, y_valid, "test")

# Further testing set
# testloader2 = prepare_dataloader(human_action_dataset, labels, "test2")

In [4]:
from tensorflow.keras.utils import register_keras_serializable

@register_keras_serializable(package="Custom", name="TubeletEmbedding")
class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim, # embed_dim = 64
            kernel_size=patch_size, # patch_size = (8, 8, 8)
            strides=patch_size, 
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

@register_keras_serializable(package="Custom", name="PositionalEncoder")
class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape # input_shape =(numVid,3920,64)
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1) # Set positions

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions # Concat the position with it token
        return encoded_tokens

In [ ]:
# '''
# Params:
#   Added drop out layers
# '''
# def create_vivit_classifier(
#     tubelet_embedder,
#     positional_encoder,
#     input_shape=INPUT_SHAPE,
#     transformer_layers=NUM_LAYERS,
#     num_heads=NUM_HEADS,
#     embed_dim=PROJECTION_DIM,
#     layer_norm_eps=LAYER_NORM_EPS,
#     num_classes=NUM_CLASSES,
#     dropout_rate=0.3,
# ):
#     # Get the input layer
#     inputs = layers.Input(shape=input_shape)
#     # Create patches.
#     patches = tubelet_embedder(inputs)
#     # Encode patches.
#     encoded_patches = positional_encoder(patches)

#     # Create multiple layers of the Transformer block.
#     for _ in range(transformer_layers):
#         # Layer normalization and MHSA
#         x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
#         attention_output = layers.MultiHeadAttention(
#             num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
#         )(x1, x1)

#         #NOTE: Adding drop out for attention output
#         attention_output = layers.Dropout(dropout_rate)(attention_output)

#         # Skip connection
#         x2 = layers.Add()([attention_output, encoded_patches])

#         # Layer Normalization and MLP
#         x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
#         x3 = keras.Sequential(
#             [
#                 layers.Dense(units=embed_dim * 4, activation=ops.gelu),
#                 layers.Dense(units=embed_dim, activation=ops.gelu),
#             ]
#         )(x3)

#         #NOTE Dropout after MLP
#         x3 = layers.Dropout(dropout_rate)(x3)

#         # Skip connection
#         encoded_patches = layers.Add()([x3, x2])

#     # Layer normalization and Global average pooling.
#     representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
#     representation = layers.GlobalAvgPool1D()(representation)

#     #NOTE: Optional dropout after global average pooling
#     representation = layers.Dropout(dropout_rate)(representation)

#     # Classify outputs.
#     outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

#     # Create the Keras model.
#     model = keras.Model(inputs=inputs, outputs=outputs)
#     return model

In [13]:
# Base line
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,# (40, 224, 224, 3)
    transformer_layers=NUM_LAYERS,# num_layers = 2
    num_heads=NUM_HEADS, # num_heads = 2
    embed_dim=PROJECTION_DIM,# embed_dim = 64
    layer_norm_eps=LAYER_NORM_EPS, # le-6
    num_classes=NUM_CLASSES,# 2(normal, abnormal)
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

        # Multi-head
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1 # NOTE: dropout
        )(x1, x1)

        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        
        # MLP
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=ops.gelu),
                layers.Dense(units=embed_dim, activation=ops.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Final ViViT Model
NOTE: 
1. This was run after the hyperparameter completed
2. Findings: Drop out rate = 0.4 & learning rate = 1e-4 had score the highes val_accuracy

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined create_vivit_classifier, trainloader, validloader, and testloader

EPOCHS = 40

def run_experiment():
    # Initialize mirrored strategy
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

    # Create and compile model within strategy scope
    with strategy.scope():
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

    # Train the model
    history = model.fit(
        trainloader,
        epochs=EPOCHS,
        validation_data=validloader,
        callbacks=[early_stopping],
    )

    # Evaluate on test set
    print("--------------------------------------------------")
    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history, model

history, model = run_experiment()


## Evaluation

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp

# Sample data (replace these with your actual loss values)
loss2 = history.history['loss']
val_loss = history.history['val_loss']

epochs = list(range(1, len(loss2) + 1))

# Create subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Accuracy', 'Loss'))

# Remove accuracy subplot by creating only one subplot for loss
fig = sp.make_subplots(rows=1, cols=1, subplot_titles=('Loss',))

# Add traces for loss
fig.add_trace(
    go.Scatter(x=epochs, y=loss2, mode='lines', name='Train Loss', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=val_loss, mode='lines', name='Test Loss', line=dict(color='red')),
    row=1, col=1
)

# Update layout
fig.update_layout(
    title_text='ViViT Training and Validation Loss',
    showlegend=True,
    xaxis_title='Epoch',
    yaxis_title='Value'
)

# Update xaxis and yaxis titles for the loss subplot
fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_yaxes(title_text='Loss', dtick=0.1, row=1, col=1)  # Change dtick to desired step size for Loss

# Show figure
fig.show()

### Other metrics

In [ ]:
 _, accuracy, _ = model.evaluate(testloader)

In [ ]:
from sklearn.metrics import classification_report
def calculate_metrics(model, testloader):
    # Get the true labels and predictions
    y_true = []
    y_pred = []
    label_dict = {0: 'Normal', 1: 'Abnormal'}

    for frames, labels in testloader:
        preds = model.predict(frames)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    # Calculate metrics
    report = classification_report(y_true, y_pred, target_names=label_dict.values(), output_dict=True)
    
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    f1_score = report['weighted avg']['f1-score']

    return precision, recall, f1_score


precision, recall, f1_score = calculate_metrics(model,testloader)
print(f"Accuracy: {round(accuracy * 100, 2)}%")
print(f'Precision: {round(precision * 100,2)}%')
print(f'Recall: {round(recall * 100,2)}%')
print(f'f1_score: {round(f1_score * 100,2)}%')

### Confusion Matrix

In [ ]:
# Get the true labels
true_labels = np.concatenate([y for x, y in testloader], axis=0)

# Get the predictions
predictions = model.predict(testloader)
# predictions = model.predict(validloader)
predicted_labels = np.argmax(predictions, axis=1)
z = tf.math.confusion_matrix(labels=true_labels, predictions=predicted_labels)

# Plot matrix
import plotly.express as px
fig = px.imshow(z,text_auto=True,color_continuous_scale='blues')

fig.update_layout(
    title='Confusion Matrix - Test Set',
    xaxis_title='Predicted Labels',
    yaxis_title='True Labels',
    width=800,  # Set width of the figure
    height=600,  # Set height of the figure
)
    
fig.show()

# Model Before Tuned
- Not tuned model

NOTE: Accuracy score was good however the fluctuation of the vlidation loss is terrible

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined create_vivit_classifier, trainloader, validloader, and testloader

EPOCHS = 40

def run_experiment():
    # Initialize mirrored strategy
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

    # Create and compile model within strategy scope
    with strategy.scope():
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
        )

        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

    # Callbacks for early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

    # Train the model
    history = model.fit(
        trainloader,
        epochs=EPOCHS,
        validation_data=validloader,
        callbacks=[early_stopping],
    )

    # Evaluate on test set
    print("--------------------------------------------------")
    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%") #NOTE: If the true label under the top 5 then consider correct prediction

    return history, model

history, model = run_experiment()


## Loss Graph

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp

# Sample data (replace these with your actual loss values)
loss2 = history.history['loss']
val_loss = history.history['val_loss']

epochs = list(range(1, len(loss2) + 1))


# Remove accuracy subplot by creating only one subplot for loss
fig = sp.make_subplots(rows=1, cols=1, subplot_titles=('Loss',))

# Add traces for loss
fig.add_trace(
    go.Scatter(x=epochs, y=loss2, mode='lines', name='Train Loss', line=dict(color='blue')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=epochs, y=val_loss, mode='lines', name='Test Loss', line=dict(color='red')),
    row=1, col=1
)

# Update layout
fig.update_layout(
    title_text='ViViT Training and Validation Loss',
    showlegend=True,
    xaxis_title='Epoch',
    yaxis_title='Value'
)

# Update xaxis and yaxis titles for the loss subplot
fig.update_xaxes(title_text='Epoch', row=1, col=1)
fig.update_yaxes(title_text='Loss', dtick=0.1, row=1, col=1)  # Change dtick to desired step size for Loss

# Show figure
fig.show()

## Accuracy graph

In [ ]:
# Average trian and validation accuracy

acc2 = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Calculate the average accuracy
avg_train_acc = np.mean(acc2)
avg_val_acc = np.mean(val_acc)

print(f"Average Train Accuracy: {avg_train_acc*100}%")
print(f"Average Validation Accuracy: {avg_val_acc*100}%")


# Hyperparameter tuning

In [ ]:
# Base line
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    dropout_rate,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=dropout_rate
        )(x1, x1)


        # Skip connection
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = keras.Sequential(
            [
                layers.Dense(units=embed_dim * 4, activation=ops.gelu),
                layers.Dense(units=embed_dim, activation=ops.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined create_vivit_classifier, trainloader, validloader, and testloader

EPOCHS = 40

def build_model(hp):
    # Initialize mirrored strategy
    strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
    print(f"Number of devices: {strategy.num_replicas_in_sync}")

    # Create and compile model within strategy scope
    DROPOUT_RATE = hp.Float('dropout_rate', min_value=0.2, max_value=0.8, step=0.1) #NOTE: Total 7
    
    with strategy.scope():
        model = create_vivit_classifier(
            tubelet_embedder=TubeletEmbedding(
                embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
            ),
            positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
            dropout_rate = DROPOUT_RATE,
        )
        
        LEARNING_RATE = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5]) #NOTE: total 4
        optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=[
                keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
                keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
            ],
        )

    return model
#  model = run_experiment()


In [ ]:
tuner = kt.GridSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=50,
    executions_per_trial=1,
    overwrite=True,
    directory='my_dir',
    project_name="ViViT_GridSearch",
)

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(trainloader, epochs=40, validation_data=validloader)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""The hyperparameter grid serachis completed.The optimal parameter for
dropput is {best_hps.get('dropout_rate')} and for the learnign rate is {best_hps.get('learning_rate')}""")

# Save model

In [ ]:
model.save('ViViT_24Jul.keras')

# Demostration

In [5]:
model = tf.keras.models.load_model('../Saved_model/ViViT_3July_2.keras')

/Users/linzhanyao/opt/anaconda3/envs/tensor_env/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'tubelet_embedding', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [21]:
import cv2
import numpy as np
import tensorflow as tf

def load_all_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return {'frames': None, 'frames_dim': None, 'success': False}

    frames_dims = []
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        h, w, c = frame.shape
        frames_dims.append([0, h, w, c])
        frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_CUBIC)
        frames.append(frame)

    cap.release()
    return {'frames': np.asarray(frames), 'frames_dim': frames_dims, 'success': True}

def trim_video_frames(video, max_frame):
    '''
    Args:
        video: video (collection of frames)
        max_frame: max number of frames
    '''
    f, _, _, _ = video.shape
    startf = f // 2 - max_frame // 2
    return video[startf:startf + max_frame, :, :, :]

def preprocess_single_video(video):
    video = trim_video_frames(video, 40)
    video = tf.image.convert_image_dtype(video, tf.float32)
    return video

label_dict = {0: 'Normal', 1: 'Abnormal'}
video_path = '../Dataset/Test_dataset/abnormal/video_254_flip.avi'

load_data = load_all_frames(video_path)
if not load_data['success']:
    print('Video is corrupt!!')
else:
    video = load_data['frames']
    preprocessed_video = preprocess_single_video(video)
    output = model.predict(tf.expand_dims(preprocessed_video, axis=0))[0]
    print(output)
    pred = np.argmax(output, axis=0)
    print(label_dict[pred])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step
[3.021496e-04 9.996979e-01]
Abnormal
